In [0]:
spark.conf.set(
    "fs.azure.account.key.bp040storage.dfs.core.windows.net",
    "kiENG28kPnyzB+dA3xwAxlcWC5eHv56ldDJidJPlneDLglk3zhWyPkLZL+TOOzfqiyFn7i9Ys7x3+AStxKkaaQ==")

In [0]:
display(dbutils.fs.ls("abfss://bp040container@bp040storage.dfs.core.windows.net"))

path,name,size,modificationTime
abfss://bp040container@bp040storage.dfs.core.windows.net/raw/,raw/,0,1729970244000


In [0]:
df1 = spark.read.csv("abfss://bp040container@bp040storage.dfs.core.windows.net/raw/sales_data.csv", header=True, inferSchema="true");
df1.show(5)

+-------+-----------+-------------------+----------+------------+---------+----------+-----------+
|OrderID| CustomerID|          OrderDate| ProductID| ProductName| Quantity| UnitPrice| TotalPrice|
+-------+-----------+-------------------+----------+------------+---------+----------+-----------+
|      1|     1001.0|2022-01-01 00:00:00|       1.0|      Laptop|      2.0|    1000.0|     2000.0|
|      2|     1002.0|2022-01-02 00:00:00|       2.0|  Smartphone|      3.0|     800.0|     2400.0|
|      3|     1003.0|2022-01-03 00:00:00|       1.0|      Laptop|      1.0|    1000.0|     1000.0|
|      4|     1001.0|2022-01-04 00:00:00|       3.0|  Headphones|      2.0|      50.0|      100.0|
|      5|     1002.0|2022-01-05 00:00:00|       1.0|      Laptop|      2.0|    1000.0|     2000.0|
+-------+-----------+-------------------+----------+------------+---------+----------+-----------+
only showing top 5 rows



In [0]:
from pyspark.sql.functions import *

df2 = df1.withColumn("year",year(" OrderDate")) \
       .withColumn("month",month(" OrderDate")) \
       .withColumn("day",dayofmonth(" OrderDate"))
df2.show(5)

+-------+-----------+-------------------+----------+------------+---------+----------+-----------+----+-----+---+
|OrderID| CustomerID|          OrderDate| ProductID| ProductName| Quantity| UnitPrice| TotalPrice|year|month|day|
+-------+-----------+-------------------+----------+------------+---------+----------+-----------+----+-----+---+
|      1|     1001.0|2022-01-01 00:00:00|       1.0|      Laptop|      2.0|    1000.0|     2000.0|2022|    1|  1|
|      2|     1002.0|2022-01-02 00:00:00|       2.0|  Smartphone|      3.0|     800.0|     2400.0|2022|    1|  2|
|      3|     1003.0|2022-01-03 00:00:00|       1.0|      Laptop|      1.0|    1000.0|     1000.0|2022|    1|  3|
|      4|     1001.0|2022-01-04 00:00:00|       3.0|  Headphones|      2.0|      50.0|      100.0|2022|    1|  4|
|      5|     1002.0|2022-01-05 00:00:00|       1.0|      Laptop|      2.0|    1000.0|     2000.0|2022|    1|  5|
+-------+-----------+-------------------+----------+------------+---------+----------+--

In [0]:
df2.write.mode("overwrite").partitionBy(" OrderDate").format("parquet").save("abfss://processed1@bp040storage.dfs.core.windows.net/bcd/partition/sales")

In [0]:
df2.write.mode("overwrite").partitionBy("year","month","day").format("parquet").save("abfss://processed1@bp040storage.dfs.core.windows.net/bcd/partition/sales")

In [0]:
spark.read.format("parquet").load("abfss://processed1@bp040storage.dfs.core.windows.net/bcd/partition/sales").show()

+-------+-----------+-------------------+----------+------------+---------+----------+-----------+----+-----+---+
|OrderID| CustomerID|          OrderDate| ProductID| ProductName| Quantity| UnitPrice| TotalPrice|year|month|day|
+-------+-----------+-------------------+----------+------------+---------+----------+-----------+----+-----+---+
|      9|     1003.0|2022-02-04 00:00:00|       2.0|  Smartphone|      2.0|     800.0|     1600.0|2022|    2|  4|
|      8|     1002.0|2022-02-03 00:00:00|       3.0|  Headphones|      1.0|      50.0|       50.0|2022|    2|  3|
|      4|     1001.0|2022-01-04 00:00:00|       3.0|  Headphones|      2.0|      50.0|      100.0|2022|    1|  4|
|      6|     1003.0|2022-02-01 00:00:00|       2.0|  Smartphone|      4.0|     800.0|     3200.0|2022|    2|  1|
|      2|     1002.0|2022-01-02 00:00:00|       2.0|  Smartphone|      3.0|     800.0|     2400.0|2022|    1|  2|
|      3|     1003.0|2022-01-03 00:00:00|       1.0|      Laptop|      1.0|    1000.0|  